In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
from tqdm import tqdm

2025-09-08 15:40:29.960534: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# CHIẾN LƯỢC TĂNG CƯỜNG DỮ LIỆU CHO RICE LEAF DISEASE

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random

# Định nghĩa lại các biến cần thiết
DATA_DIR = Path("data/splits")
PROCESSED_DIR = Path("data/processed")
AUGMENTED_DIR = Path("data/augmented")
CLASSES_CORRECTED = ["brown_spot", "blast", "bacterial_leaf_blight", "normal"]
SPLITS = ["train", "val", "test"]
TARGET_SIZE = (224, 224)

def create_augmentation_strategy():
    """Tạo chiến lược tăng cường dữ liệu phù hợp với ảnh bệnh lá lúa"""
    
    # Augmentation cho training set
    train_datagen = ImageDataGenerator(
        # Biến đổi hình học
        rotation_range=20,           # Xoay ±20 độ (lá lúa có thể nghiêng)
        width_shift_range=0.1,       # Dịch chuyển ngang 10%
        height_shift_range=0.1,      # Dịch chuyển dọc 10%
        shear_range=0.1,             # Biến dạng cắt 10%
        zoom_range=0.1,              # Zoom ±10%
        horizontal_flip=True,        # Lật ngang (lá lúa đối xứng)
        vertical_flip=False,         # Không lật dọc (lá lúa có hướng)
        
        # Biến đổi màu sắc (quan trọng cho ảnh bệnh)
        brightness_range=[0.8, 1.2], # Thay đổi độ sáng ±20%
        channel_shift_range=20,      # Thay đổi màu sắc
        fill_mode='nearest',         # Điền pixel gần nhất
        
        # Chuẩn hóa
        rescale=1./255,              # Chuẩn hóa về [0,1]
    )
    
    # Augmentation cho validation/test (ít biến đổi hơn)
    val_datagen = ImageDataGenerator(
        rescale=1./255,
        # Chỉ thêm một chút biến đổi nhẹ
        brightness_range=[0.9, 1.1],
    )
    
    return train_datagen, val_datagen

def analyze_class_distribution():
    """Phân tích phân bố lớp để xác định cần tăng cường bao nhiêu cho tất cả splits"""
    class_counts = {}
    
    for split in SPLITS:
        class_counts[split] = {}
        for cls in CLASSES_CORRECTED:
            path = PROCESSED_DIR / split / cls
            if path.exists():
                count = len(list(path.glob("*")))
                class_counts[split][cls] = count
            else:
                class_counts[split][cls] = 0
    
    # Hiển thị thống kê
    print("=== PHÂN BỐ DỮ LIỆU SAU TIỀN XỬ LÝ ===")
    for split in SPLITS:
        print(f"\n{split.upper()}:")
        for cls in CLASSES_CORRECTED:
            count = class_counts[split][cls]
            print(f"  {cls}: {count} ảnh")
    
    # Tính số ảnh cần tăng cường cho từng split
    # Tìm số ảnh tối đa trong mỗi split
    max_counts = {}
    for split in SPLITS:
        max_counts[split] = max(class_counts[split].values())
    
    # Tạo kế hoạch tăng cường cho từng split
    augmentation_plan = {}
    for split in SPLITS:
        augmentation_plan[split] = {}
        target_count = max_counts[split] * 2  # Tăng gấp đôi cho mỗi split
        
        print(f"\n=== KẾ HOẠCH TĂNG CƯỜNG {split.upper()} ===")
        print(f"Số ảnh tối đa hiện tại: {max_counts[split]}")
        print(f"Mục tiêu tăng cường: {target_count} ảnh/lớp")
        
        for cls in CLASSES_CORRECTED:
            current = class_counts[split][cls]
            needed = target_count - current
            augmentation_plan[split][cls] = max(0, needed)
            print(f"  {cls}: {current} → {target_count} (+{needed})")
    
    return class_counts, augmentation_plan

# Phân tích dữ liệu hiện tại
class_counts, augmentation_plan = analyze_class_distribution()


=== PHÂN BỐ DỮ LIỆU SAU TIỀN XỬ LÝ ===

TRAIN:
  brown_spot: 2116 ảnh
  blast: 1879 ảnh
  bacterial_leaf_blight: 2390 ảnh
  normal: 1869 ảnh

VAL:
  brown_spot: 459 ảnh
  blast: 406 ảnh
  bacterial_leaf_blight: 511 ảnh
  normal: 411 ảnh

TEST:
  brown_spot: 447 ảnh
  blast: 405 ảnh
  bacterial_leaf_blight: 510 ảnh
  normal: 424 ảnh

=== KẾ HOẠCH TĂNG CƯỜNG TRAIN ===
Số ảnh tối đa hiện tại: 2390
Mục tiêu tăng cường: 4780 ảnh/lớp
  brown_spot: 2116 → 4780 (+2664)
  blast: 1879 → 4780 (+2901)
  bacterial_leaf_blight: 2390 → 4780 (+2390)
  normal: 1869 → 4780 (+2911)

=== KẾ HOẠCH TĂNG CƯỜNG VAL ===
Số ảnh tối đa hiện tại: 511
Mục tiêu tăng cường: 1022 ảnh/lớp
  brown_spot: 459 → 1022 (+563)
  blast: 406 → 1022 (+616)
  bacterial_leaf_blight: 511 → 1022 (+511)
  normal: 411 → 1022 (+611)

=== KẾ HOẠCH TĂNG CƯỜNG TEST ===
Số ảnh tối đa hiện tại: 510
Mục tiêu tăng cường: 1020 ảnh/lớp
  brown_spot: 447 → 1020 (+573)
  blast: 405 → 1020 (+615)
  bacterial_leaf_blight: 510 → 1020 (+510)
  norma

In [3]:
def generate_augmented_data():
    """Tạo dữ liệu tăng cường và lưu vào thư mục augmented cho tất cả splits"""
    train_datagen, val_datagen = create_augmentation_strategy()
    
    total_generated = 0
    
    # Tạo thư mục đích nếu chưa tồn tại
    for split in SPLITS:
        for cls in CLASSES_CORRECTED:
            dst_dir = AUGMENTED_DIR / split / cls
            dst_dir.mkdir(parents=True, exist_ok=True)
    
    # Xử lý từng split
    for split in SPLITS:
        print(f"\n{'='*50}")
        print(f"TĂNG CƯỜNG DỮ LIỆU CHO {split.upper()}")
        print(f"{'='*50}")
        
        # Chọn datagen phù hợp
        if split == 'train':
            datagen = train_datagen
        else:
            datagen = val_datagen  # Sử dụng val_datagen cho val và test (ít biến đổi hơn)
        
        for cls in CLASSES_CORRECTED:
            src_dir = PROCESSED_DIR / split / cls
            dst_dir = AUGMENTED_DIR / split / cls
            
            if not src_dir.exists():
                print(f"Thư mục không tồn tại: {src_dir}")
                continue
            
            # Đếm số ảnh hiện tại
            current_count = len(list(src_dir.glob("*")))
            target_count = augmentation_plan[split][cls]
            
            if target_count <= 0:
                print(f"{split}/{cls}: Đã đủ ảnh ({current_count})")
                continue
            
            print(f"\nTăng cường {split}/{cls}: {current_count} → {current_count + target_count}")
            
            # Tạo generator
            generator = datagen.flow_from_directory(
                PROCESSED_DIR / split,
                target_size=TARGET_SIZE,
                batch_size=1,
                class_mode='categorical',
                classes=[cls],
                save_to_dir=str(dst_dir),
                save_prefix=f'aug_{split}_{cls}',
                save_format='jpg',
                shuffle=True
            )
            
            # Tạo ảnh tăng cường
            for i in range(target_count):
                batch = next(generator)
                total_generated += 1
                
                if (i + 1) % 50 == 0:  # Giảm tần suất in cho val/test
                    print(f"  Đã tạo {i + 1}/{target_count} ảnh...")
    
    print(f"\n=== HOÀN THÀNH TĂNG CƯỜNG ===")
    print(f"Tổng số ảnh đã tạo: {total_generated}")
    
    # Kiểm tra kết quả cho tất cả splits
    print("\n=== KIỂM TRA KẾT QUẢ ===")
    for split in SPLITS:
        print(f"\n{split.upper()}:")
        for cls in CLASSES_CORRECTED:
            original_count = len(list((PROCESSED_DIR / split / cls).glob("*")))
            augmented_count = len(list((AUGMENTED_DIR / split / cls).glob("*")))
            print(f"  {cls}: {original_count} gốc + {augmented_count} tăng cường = {original_count + augmented_count} tổng")

# Chạy tăng cường dữ liệu
print("=== BẮT ĐẦU TĂNG CƯỜNG DỮ LIỆU CHO TẤT CẢ SPLITS ===")
generate_augmented_data()


=== BẮT ĐẦU TĂNG CƯỜNG DỮ LIỆU CHO TẤT CẢ SPLITS ===

TĂNG CƯỜNG DỮ LIỆU CHO TRAIN

Tăng cường train/brown_spot: 2116 → 4780
Found 2116 images belonging to 1 classes.
  Đã tạo 50/2664 ảnh...
  Đã tạo 100/2664 ảnh...
  Đã tạo 150/2664 ảnh...
  Đã tạo 200/2664 ảnh...
  Đã tạo 250/2664 ảnh...
  Đã tạo 300/2664 ảnh...
  Đã tạo 350/2664 ảnh...
  Đã tạo 400/2664 ảnh...
  Đã tạo 450/2664 ảnh...
  Đã tạo 500/2664 ảnh...
  Đã tạo 550/2664 ảnh...
  Đã tạo 600/2664 ảnh...
  Đã tạo 650/2664 ảnh...
  Đã tạo 700/2664 ảnh...
  Đã tạo 750/2664 ảnh...
  Đã tạo 800/2664 ảnh...
  Đã tạo 850/2664 ảnh...
  Đã tạo 900/2664 ảnh...
  Đã tạo 950/2664 ảnh...
  Đã tạo 1000/2664 ảnh...
  Đã tạo 1050/2664 ảnh...
  Đã tạo 1100/2664 ảnh...
  Đã tạo 1150/2664 ảnh...
  Đã tạo 1200/2664 ảnh...
  Đã tạo 1250/2664 ảnh...
  Đã tạo 1300/2664 ảnh...
  Đã tạo 1350/2664 ảnh...
  Đã tạo 1400/2664 ảnh...
  Đã tạo 1450/2664 ảnh...
  Đã tạo 1500/2664 ảnh...
  Đã tạo 1550/2664 ảnh...
  Đã tạo 1600/2664 ảnh...
  Đã tạo 1650/2664 ản

In [4]:
def verify_augmented_structure():
    """Kiểm tra cấu trúc thư mục sau khi tăng cường"""
    print("=== KIỂM TRA CẤU TRÚC THƯ MỤC AUGMENTED ===")
    
    for split in SPLITS:
        print(f"\n{split.upper()}:")
        split_dir = AUGMENTED_DIR / split
        if split_dir.exists():
            for cls in CLASSES_CORRECTED:
                cls_dir = split_dir / cls
                if cls_dir.exists():
                    count = len(list(cls_dir.glob("*")))
                    print(f"  {cls}: {count} ảnh")
                else:
                    print(f"  {cls}: Thư mục không tồn tại")
        else:
            print(f"  Thư mục {split} không tồn tại")

# Chạy kiểm tra cấu trúc
verify_augmented_structure()


=== KIỂM TRA CẤU TRÚC THƯ MỤC AUGMENTED ===

TRAIN:
  brown_spot: 2664 ảnh
  blast: 2901 ảnh
  bacterial_leaf_blight: 2390 ảnh
  normal: 2911 ảnh

VAL:
  brown_spot: 563 ảnh
  blast: 616 ảnh
  bacterial_leaf_blight: 511 ảnh
  normal: 611 ảnh

TEST:
  brown_spot: 573 ảnh
  blast: 615 ảnh
  bacterial_leaf_blight: 510 ảnh
  normal: 596 ảnh
